In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import shapefile
import random
from haversine import haversine
import arcpy

In [5]:
#读入文件
df_raw = pd.DataFrame(pd.read_excel("data/syfhd_20220623.xls"))

In [5]:
#获取所有的mmsi
# mmsi_list = df_raw["mmsi"].unique()
# mmsi_list[:5]

# 可视化ais轨迹
# for mmsi in mmsi_list:
#     ais = df[df['mmsi']==mmsi]
#     x = ais["经度"]
#     y = ais["纬度"]


#     plt.plot(x,y)
#     plt.show()


In [6]:
# 去掉信息少于10的ais 并只保留时间、mmsi、经纬度
value_counts = df_raw['mmsi'].value_counts()
to_remove = value_counts[value_counts<=10].index
df_light = df_raw.replace(to_remove, np.nan,inplace=False)
df_light = df_light.loc[:,["updatetime","mmsi","经度","纬度"]]
df_light.dropna(inplace=True)

In [7]:
# # AIS 插值等间隔
# df = df_light
# sleep = 500 #等间隔值
# time = 0
# i = 0
# dic={'时间点(s)': [],'CPU使用率(%)':[]}
# while time<=df.iloc[-1]['时间点(s)']:
#     while time>=df.iloc[i]['时间点(s)']:
#         i+=1
#     z = df.iloc[i]['CPU使用率(%)'] - df.iloc[i-1]['CPU使用率(%)']
#     p = df.iloc[i]['时间点(s)'] - df.iloc[i-1]['时间点(s)']
#     t = time - df.iloc[i-1]['时间点(s)']
#     v = df.iloc[i-1]['CPU使用率(%)'] + z / p * t
#     dic['时间点(s)'].append(time)
#     dic['CPU使用率(%)'].append(v)
#     print(time,v)
#     time += sleep
# df1 = pd.DataFrame(dic)
# df1.to_excel('new.xlsx',index=None)
 

In [8]:
#shp文件中寻找最接近（lon,lat）的
def find_shp(lon, lat):
    sf = shapefile.Reader("data/三亚船舶标注汇总_762.shp",encoding='gb18030')
    min_diff = float("inf")
    
    shapes = sf.shapes()
    for shape in shapes:
        center_lon,center_lat = np.mean(shape.bbox[0::2]),np.mean(shape.bbox[1::2])
        
        diff = haversine((center_lat,center_lon), (lat,lon), unit='m')
        if diff<min_diff:
            min_diff = diff
            res_shape = shape
    return res_shape, min_diff


In [27]:
# 查看ais所有船的时间在图像时间是否命中
ais_g = df_light.groupby("mmsi")
start_time,time_delta_time = "2022-06-23 10:46" ,timedelta(days=1)
start_time=datetime.strptime(start_time, "%Y-%m-%d %H:%M")

min_diff_list=[]
for mmsi,content in ais_g:
    
    filter = content[(content["updatetime"]>=start_time-time_delta_time) & (content["updatetime"]<=start_time+time_delta_time)]
    if len(filter)==0:
        #预测
        pass
    else:
        Lon_mean,Lat_mean = filter["经度"].mean(),filter["纬度"].mean()
        
        p1x,p1y,p2x,p2y = find_shp(Lon_mean,Lat_mean)[0].bbox
        
        match_shp,min_diff = find_shp(Lon_mean,Lat_mean)
        min_diff_list.append(min_diff)

        ais_list = random.sample(list(df_raw[df_raw["mmsi"]==mmsi].values),5)
print(np.max(min_diff_list)) 
print(len(min_diff_list))
    

0.019517066467862776
145
